In [1]:
import pandas as pd
import numpy as np

In [3]:
pble = pd.read_csv('../../Dados/PBLE/PBLE.csv', encoding='latin-1', sep=';')

In [4]:
# removendo linha com dados inválidos
pble.drop(27192, inplace=True)

pble['Ano_Ativacao'] = pble['Data de Ativação'].apply(lambda x: int(x[6:]))

In [5]:
pble=pble[pble['Ano_Ativacao']>=2018].sort_values(by='Ano_Ativacao')

In [6]:
pble.head()

,Prestadora,Nº INEP,UF,Municipio,Situação,Tipo de Obrigação,Velocidade de Acesso Instalada,Tecnologia,Nome da Escola,Tipo Escola,Data de Ativação,Mes,Ano,Ano_Ativacao
51271,TELEFÔNICA,35205187[*],SP,Campinas,Instalada,PBLE ...,10 Mbps,ADSL,JOAO VIALTA CEI,Municipal,15/10/2018,3,2022,2018
19850,TELEFÔNICA,35910284,SP,São Paulo,Instalada,PBLE ...,2 Mbps,DEDICADO,SERGIO PAULO MUNIZ PIMENTA CAPITAO,Estadual,22/03/2018,3,2022,2018
19855,TELEFÔNICA,35037497,SP,Guarulhos,Instalada,PBLE ...,2 Mbps,ADSL,BENEDITA DE OLIVEIRA ALE PROFESSORA,Estadual,23/02/2018,3,2022,2018
19872,TELEFÔNICA,35003463,SP,São Paulo,Instalada,PBLE ...,2 Mbps,DEDICADO,REINALDO RIBEIRO DA SILVA DOUTOR,Estadual,19/02/2018,3,2022,2018
19966,TELEFÔNICA,35565817[*],SP,Caraguatatuba,Instalada,PBLE ...,8 Mbps,Rede Celular,LUIZ SILVAR DO PRADO PROFESSOR EMEF,Municipal,05/09/2018,3,2022,2018


In [7]:
pble['velocidade'] = pble['Velocidade de Acesso Instalada'].apply(lambda x: float(x.split(' ')[0]))

In [8]:
pble[['Velocidade de Acesso Instalada', 'velocidade']]

,Velocidade de Acesso Instalada,velocidade
51271,10 Mbps,10.0
19850,2 Mbps,2.0
19855,2 Mbps,2.0
19872,2 Mbps,2.0
19966,8 Mbps,8.0
...,...,...
61705,2 Mbps,2.0
45140,25 Mbps,25.0
40548,25 Mbps,25.0
33122,25 Mbps,25.0


In [48]:
pble_agg_tipo_escola = pble.groupby(['Ano_Ativacao','Municipio','Tipo Escola']).agg(velocidade_median=('velocidade', np.median), velocidade_mean=('velocidade', np.mean)).reset_index()

In [49]:
pble_agg_total_municipio = pble.groupby(['Ano_Ativacao','Municipio']).agg(velocidade_median=('velocidade', np.median), velocidade_mean=('velocidade', np.mean)).reset_index()

### Salvando agregações

In [54]:
pble_agg_total_municipio.to_csv('../../Resultados/Agg/PBLE/PBLE_agg_total_municipio.csv', index=False)

In [55]:
pble_agg_tipo_escola.to_csv('../../Resultados/Agg/PBLE/PBLE_agg_tipo_escola.csv', index=False)

### Investigando correspondência CENSO Escolar x PBLE

In [34]:
import difflib 

In [23]:
cols_list = [
    'NU_ANO_CENSO',
    'SG_UF',
    'CO_MUNICIPIO',
    'NO_MUNICIPIO',
    'CO_ENTIDADE',
    'NO_ENTIDADE',
    'TP_DEPENDENCIA',
    'TP_LOCALIZACAO',
    'IN_BAS',
    'IN_MED',
    'TP_SITUACAO_FUNCIONAMENTO',
    'IN_BANDA_LARGA',
]

In [24]:
years = [str(i) for i in range(2018, 2021+1)]

In [25]:
def open_dfs(years):
    dfs = []
    for year in years:
        censo = pd.read_csv(
        f'../../Dados/Censo Escolar/{year}/dados/microdados_ed_basica_{year}.csv', 
        sep=';', 
        encoding='latin-1',
        usecols=cols_list,
        #assume_missing=True,
        dtype={'CO_MUNICIPIO': 'object',
                'CO_ENTIDADE': 'object',
                'NU_ANO_CENSO': 'int64'}
        )
        dfs.append(censo)
    return dfs

In [26]:
all = pd.concat(open_dfs(years))

In [29]:
pble[~pble['Nome da Escola'].isin(all.NO_ENTIDADE)]

,Prestadora,Nº INEP,UF,Municipio,Situação,Tipo de Obrigação,Velocidade de Acesso Instalada,Tecnologia,Nome da Escola,Tipo Escola,Data de Ativação,Mes,Ano,Ano_Ativacao,velocidade
50647,TELEMAR,33167397,RJ,Armação dos Búzios,Instalada,-- ...,10 Mbps,Fibra - FTTx,INSTITUTO EDUCACIONAL DE HABILITACAO PROFISSIO...,Municipal,13/04/2018,3,2022,2018,10.0
42334,TELEMAR,15103030,PA,Santarém,Instalada,-- ...,2 Mbps,SATÉLITE,E M E F FRANCISCA DAS CHAGAS NASCIMENTO,Municipal,13/09/2018,3,2022,2018,2.0
50437,TELEFÔNICA,35435119,SP,Jundiaí,Instalada,PBLE ...,4 Mbps,ADSL,RESIDENCIAL JUNDIAI,Estadual,14/03/2018,3,2022,2018,4.0
34214,TELEFÔNICA,35446038,SP,Piracicaba,Instalada,PBLE ...,4 Mbps,DEDICADO,JARDIM DOS MANACAS,Estadual,04/09/2018,3,2022,2018,4.0
19098,TELEMAR,MG01076686,MG,Cláudio,Instalada,-- ...,2 Mbps,ADSL,NTM - CLAUDIO,Municipal,20/11/2018,3,2022,2018,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,TELEFÔNICA,35099442,SP,Cotia,Instalada,PBLE ...,25 Mbps,Fibra - FTTx,FRANCISCO DE PAULA CARRARO CENTRO EDUC.,Municipal,28/01/2022,3,2022,2022,25.0
62290,TELEFÔNICA,35220361,SP,Cotia,Instalada,PBLE ...,25 Mbps,Fibra - FTTx,JARDIM SABIA E.M.DO,Municipal,16/02/2022,3,2022,2022,25.0
9447,TELEFÔNICA,35043850,SP,Cotia,Instalada,PBLE ...,25 Mbps,Fibra - FTTx,CARLOS DIAS DE SANT ANNA PROF,Estadual,04/03/2022,3,2022,2022,25.0
38788,TELEFÔNICA,35024193,SP,Ribeirão Preto,Instalada,PBLE ...,25 Mbps,Fibra - FTTx,JOAO RODRIGUES GUIAO DR,Estadual,24/03/2022,3,2022,2022,25.0


In [33]:
all[all['NO_ENTIDADE'].isin(pble['Nome da Escola'])].drop_duplicates(subset='NO_ENTIDADE',keep='last').IN_MED.value_counts()

1.0    801
0.0    789
Name: IN_MED, dtype: int64

In [37]:
pble['Nome da Escola'] = pble['Nome da Escola'].apply(lambda x: difflib.get_close_matches(x, all['NO_ENTIDADE'])[0])

IndexError: list index out of range

In [36]:
df2.index = df2.index.map(lambda x: difflib.get_close_matches(x, df1.index)[0])

IndexError: list index out of range

In [16]:
pble

,Prestadora,Nº INEP,UF,Municipio,Situação,Tipo de Obrigação,Velocidade de Acesso Instalada,Tecnologia,Nome da Escola,Tipo Escola,Data de Ativação,Mes,Ano,Ano_Ativacao,velocidade
51271,TELEFÔNICA,35205187[*],SP,Campinas,Instalada,PBLE ...,10 Mbps,ADSL,JOAO VIALTA CEI,Municipal,15/10/2018,3,2022,2018,10.0
19850,TELEFÔNICA,35910284,SP,São Paulo,Instalada,PBLE ...,2 Mbps,DEDICADO,SERGIO PAULO MUNIZ PIMENTA CAPITAO,Estadual,22/03/2018,3,2022,2018,2.0
19855,TELEFÔNICA,35037497,SP,Guarulhos,Instalada,PBLE ...,2 Mbps,ADSL,BENEDITA DE OLIVEIRA ALE PROFESSORA,Estadual,23/02/2018,3,2022,2018,2.0
19872,TELEFÔNICA,35003463,SP,São Paulo,Instalada,PBLE ...,2 Mbps,DEDICADO,REINALDO RIBEIRO DA SILVA DOUTOR,Estadual,19/02/2018,3,2022,2018,2.0
19966,TELEFÔNICA,35565817[*],SP,Caraguatatuba,Instalada,PBLE ...,8 Mbps,Rede Celular,LUIZ SILVAR DO PRADO PROFESSOR EMEF,Municipal,05/09/2018,3,2022,2018,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61705,OI,42154634[*],SC,Santa Terezinha do Progresso,Instalada,-- ...,2 Mbps,ADSL,UD DE SANTA TEREZINHA DO PROGRESSO,Estadual,05/02/2022,3,2022,2022,2.0
45140,TELEFÔNICA,35429533,SP,Cruzeiro,Instalada,PBLE ...,25 Mbps,Fibra - FTTx,ARNOLFO AZEVEDO DR EMEF,Municipal,25/03/2022,3,2022,2022,25.0
40548,TELEFÔNICA,35010595,SP,Cotia,Instalada,PBLE ...,25 Mbps,Fibra - FTTx,BATISTA CEPELOS,Estadual,27/01/2022,3,2022,2022,25.0
33122,TELEFÔNICA,35218595,SP,Guaratinguetá,Instalada,PBLE ...,25 Mbps,Fibra - FTTx,JOSE BENEDITO AVERALDO GALHARDO PROF EMEF,Municipal,25/03/2022,3,2022,2022,25.0
